## Unemployment Rate

In [1]:
import pandas as pd
import numpy as np

In [2]:
def clean_data(path_ue, country_code, fred_code):
    
    df_ue = pd.read_csv(path_ue)
    df_ue = df_ue.rename(columns={fred_code: f'{country_code}_UE', 'DATE': 'index'})

    df_ue['index'] = pd.to_datetime(df_ue['index'])

    df_ue['year'] = df_ue['index'].dt.year
    df_ue['month'] = df_ue['index'].dt.month

    df_ue[f'{country_code}_UE'] = df_ue[f'{country_code}_UE']/100

    return df_ue

In [3]:
def data_combine(path, df_ue):
    
    exchange_df = pd.read_csv(path) #still 4997rows
    print(exchange_df.shape)

    df_with_ue = pd.merge(exchange_df, df_ue, left_on=['month', 'year'], right_on=['month', 'year'])
    df_with_ue.drop(columns='index', inplace=True)

    return df_with_ue

In [53]:
def if_quarterly(df):

    test = pd.read_csv('/Users/zumiis/final_not4git/interest rate/switzerland from fred.csv')
    test['DATE'] = pd.to_datetime(test['DATE'])
    test['month'] = test['DATE'].dt.month
    test['year'] = test['DATE'].dt.year
    test = test.rename(columns={'IRSTCI01CHM156N': 'CHF_IR', 'DATE': 'index'})

    merged_df = pd.merge(test, df, left_on=['index'], right_on=['index'], how='outer').interpolate()#.ffill().bfill()

    merged_df.drop(columns= ['CHF_IR', 'month_y', 'year_y'], inplace=True)
    merged_df.rename(columns={'month_x':'month', 'year_x':'year'}, inplace=True)

    return merged_df


In [195]:
path_ue = '/Users/zumiis/final_not4git/unemployment/usa LRHUTTTTUSM156S.csv'
country_code = 'USD'
fred_code = 'LRHUTTTTUSM156S'

In [196]:
df_ue = clean_data(path_ue, country_code, fred_code)
df_ue

,index,USD_UE,year,month
0,2000-01-01,0.040,2000,1
1,2000-02-01,0.041,2000,2
2,2000-03-01,0.040,2000,3
3,2000-04-01,0.038,2000,4
4,2000-05-01,0.040,2000,5
...,...,...,...,...
235,2019-08-01,0.037,2019,8
236,2019-09-01,0.035,2019,9
237,2019-10-01,0.036,2019,10
238,2019-11-01,0.035,2019,11


In [169]:
df_ue_months = if_quarterly(df_ue)
df_ue_months #240 rows

,index,month,year,CHF_UE
0,2000-01-01,1,2000,0.026000
1,2000-02-01,2,2000,0.025667
2,2000-03-01,3,2000,0.025333
3,2000-04-01,4,2000,0.025000
4,2000-05-01,5,2000,0.025000
...,...,...,...,...
236,2019-08-01,8,2019,0.043085
237,2019-09-01,9,2019,0.041906
238,2019-10-01,10,2019,0.040727
239,2019-11-01,11,2019,0.040727


In [6]:
path = '/Users/zumiis/final_not4git/for github/data_with_interest_rate_gdp_trade_all.csv'
path2 = '/Users/zumiis/final_not4git/for github/data_with_interest_rate_gdp_trade_ue_all.csv'

In [197]:
#have path for the first combine instance, then use path2 - just for safe-keeping in case something goes wrong
df_with_ue = data_combine(path2, df_ue)
print(df_with_ue.shape) #4997 rows
print(df_with_ue.isna().sum())

(4997, 79)
(4997, 80)
Time Series    0
AUD_USD        0
NZD_USD        0
GBP_USD        0
BRL_USD        0
              ..
DKK_UE         0
JPY_UE         0
NOK_UE         0
SEK_UE         0
USD_UE         0
Length: 80, dtype: int64


In [199]:
df_with_ue.to_csv('/Users/zumiis/final_not4git/for github/data_with_interest_rate_gdp_trade_ue_all.csv', index=False)

In [198]:
df_with_ue

,Time Series,AUD_USD,NZD_USD,GBP_USD,BRL_USD,CND_USD,CNY_USD,IDR_USD,KRW_USD,MXN_USD,...,NZD_UE,GBP_UE,CND_UE,KRW_UE,MXN_UE,DKK_UE,JPY_UE,NOK_UE,SEK_UE,USD_UE
0,2000-01-03,1.5172,1.9033,0.6146,1.8050,1.4465,8.2798,43.55,1128.00,9.4015,...,0.065,0.057,0.068,0.051,0.023000,0.050,0.047,0.034,0.062,0.040
1,2000-01-04,1.5239,1.9238,0.6109,1.8405,1.4518,8.2799,43.55,1122.50,9.4570,...,0.065,0.057,0.068,0.051,0.023000,0.050,0.047,0.034,0.062,0.040
2,2000-01-05,1.5267,1.9339,0.6092,1.8560,1.4518,8.2798,43.55,1135.00,9.5350,...,0.065,0.057,0.068,0.051,0.023000,0.050,0.047,0.034,0.062,0.040
3,2000-01-06,1.5291,1.9436,0.6070,1.8400,1.4571,8.2797,43.55,1146.50,9.5670,...,0.065,0.057,0.068,0.051,0.023000,0.050,0.047,0.034,0.062,0.040
4,2000-01-07,1.5272,1.9380,0.6104,1.8310,1.4505,8.2794,43.55,1138.00,9.5200,...,0.065,0.057,0.068,0.051,0.023000,0.050,0.047,0.034,0.062,0.040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4992,2019-12-20,1.4489,1.5135,0.7671,4.0859,1.3170,7.0063,71.07,1160.30,18.9140,...,0.041,0.037,0.056,0.037,0.032259,0.049,0.022,0.039,0.067,0.035
4993,2019-12-23,1.4457,1.5088,0.7742,4.0621,1.3167,7.0124,71.24,1163.64,18.9310,...,0.041,0.037,0.056,0.037,0.032259,0.049,0.022,0.039,0.067,0.035
4994,2019-12-26,1.4411,1.5002,0.7688,4.0602,1.3124,6.9949,71.28,1161.18,18.9440,...,0.041,0.037,0.056,0.037,0.032259,0.049,0.022,0.039,0.067,0.035
4995,2019-12-27,1.4331,1.4919,0.7639,4.0507,1.3073,6.9954,71.45,1160.87,18.8190,...,0.041,0.037,0.056,0.037,0.032259,0.049,0.022,0.039,0.067,0.035
